In [2]:
import datetime
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
import MeCab

In [3]:
# 教師データ
data = [
    [["初めまして。"], ["初めまして。よろしくお願いします。"]],
    [["どこから来たんですか？"], ["日本から来ました。"]],
    [["日本のどこに住んでるんですか？"], ["東京に住んでいます。"]],
    [["仕事は何してますか？"], ["私は会社員です。"]],
    [["お会いできて嬉しかったです。"], ["私もです！"]],
    [["おはよう。"], ["おはようございます。"]],
    [["いつも何時に起きますか？"], ["6時に起きます。"]],
    [["朝食は何を食べますか？"], ["たいていトーストと卵を食べます。"]],
    [["朝食は毎日食べますか？"], ["たまに朝食を抜くことがあります。"]],
    [["野菜をたくさん取っていますか？"], ["毎日野菜を取るようにしています。"]],
    [["週末は何をしていますか？"], ["友達と会っていることが多いです。"]],
    [["どこに行くのが好き？"], ["私たちは渋谷に行くのが好きです。"]]
]

In [4]:
# GPUのセット

gpu = -1
if gpu >= 0: # numpyかcuda.cupyか
    xp = chainer.cuda.cupy
    chainer.cuda.get_device(gpu).use()
else:
    xp = np

In [5]:
# データ変換クラスの定義

class DataConverter:
    
    def __init__(self, batch_col_size):
        """クラスの初期化
        
        Args:
            batch_col_size: 学習時のミニバッチ単語数サイズ
        """
        self.mecab = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd") # 形態素解析器
        self.vocab = {"<eos>":0, "<unknown>": 1} # 単語辞書
        self.batch_col_size = batch_col_size
        
    def load(self, data):
        """学習時に、教師データを読み込んでミニバッチサイズに対応したNumpy配列に変換する
        
        Args:
            data: 対話データ
        """
        # 単語辞書の登録
        self.vocab = {"<eos>":0, "<unknown>": 1} # 単語辞書を初期化
        for d in data:
            sentences = [d[0][0], d[1][0]] # 入力文、返答文
            for sentence in sentences:
                sentence_words = self.sentence2words(sentence) # 文章を単語に分解する
                for word in sentence_words:
                    if word not in self.vocab:
                        self.vocab[word] = len(self.vocab)
        # 教師データのID化と整理
        queries, responses = [], []
        for d in data:
            query, response = d[0][0], d[1][0] #  エンコード文、デコード文
            queries.append(self.sentence2ids(sentence=query, train=True, sentence_type="query"))
            responses.append(self.sentence2ids(sentence=response, train=True, sentence_type="response"))
        self.train_queries = xp.vstack(queries)
        self.train_responses = xp.vstack(responses)
    
    def sentence2words(self, sentence):
        """文章を単語の配列にして返却する
        
        Args:
            sentence: 文章文字列
        """
        sentence_words = []
        for m in self.mecab.parse(sentence).split("\n"): # 形態素解析で単語に分解する
            w = m.split("\t")[0].lower() # 単語
            if len(w) == 0 or w == "eos": # 不正文字、EOSは省略
                continue
            sentence_words.append(w)
        sentence_words.append("<eos>") # 最後にvocabに登録している<eos>を代入する
        return sentence_words

    def sentence2ids(self, sentence, train=True, sentence_type="query"):
        """文章を単語IDのNumpy配列に変換して返却する
        
        Args:
            sentence: 文章文字列
            train: 学習用かどうか
            sentence_type: 学習用でミニバッチ対応のためのサイズ補填方向をクエリー・レスポンスで変更するため"query"or"response"を指定　
        Returns:
            ids: 単語IDのNumpy配列
        """
        ids = [] # 単語IDに変換して格納する配列
        sentence_words = self.sentence2words(sentence) # 文章を単語に分解する
        for word in sentence_words:
            if word in self.vocab: # 単語辞書に存在する単語ならば、IDに変換する
                ids.append(self.vocab[word])
            else: # 単語辞書に存在しない単語ならば、<unknown>に変換する
                ids.append(self.vocab["<unknown>"])
        # 学習時は、ミニバッチ対応のため、単語数サイズを調整してNumpy変換する
        if train:
            if sentence_type == "query": # クエリーの場合は前方にミニバッチ単語数サイズになるまで-1を補填する
                while len(ids) > self.batch_col_size: # ミニバッチ単語サイズよりも大きければ、ミニバッチ単語サイズになるまで先頭から削る
                    ids.pop(0)
                ids = xp.array([-1]*(self.batch_col_size-len(ids))+ids, dtype="int32")
            elif sentence_type == "response": # レスポンスの場合は後方にミニバッチ単語数サイズになるまで-1を補填する
                while len(ids) > self.batch_col_size: # ミニバッチ単語サイズよりも大きければ、ミニバッチ単語サイズになるまで末尾から削る
                    ids.pop()
                ids = xp.array(ids+[-1]*(self.batch_col_size-len(ids)), dtype="int32")
        else: # 予測時は、そのままNumpy変換する
            ids = xp.array([ids], dtype="int32")
        return ids
        
    def ids2words(self, ids):
        """予測時に、単語IDのNumpy配列を単語に変換して返却する
        
        Args:
            ids: 単語IDのNumpy配列
        Returns:
            words: 単語の配列
        """
        words = [] # 単語を格納する配列
        for i in ids: # 順番に単語IDを単語辞書から参照して単語に変換する
            words.append(list(self.vocab.keys())[list(self.vocab.values()).index(i)])
        return words

In [12]:
# モデルクラスの定義

class LSTM_Encoder(chainer.Chain):
    def __init__(self, vocab_size, embed_size, hidden_size):
        """Encoderのインスタンス化
        
        Args:
            vocab_size: 使われる単語の種類数
            embed_size: 単語をベクトル表現した際のサイズ
            hidden_size: 隠れ層のサイズ
        """
        super(LSTM_Encoder, self).__init__(
            xe = L.EmbedID(vocab_size, embed_size, ignore_label=-1),
            eh = L.Linear(embed_size, 4 * hidden_size),
            hh = L.Linear(hidden_size, 4 * hidden_size)
        )

    def __call__(self, x, c, h):
        """Encoderの計算
        
        Args:
            x: one-hotな単語
            c: 内部メモリ
            h: 隠れ層
        Returns:
            次の内部メモリ, 次の隠れ層
        """
        e = F.tanh(self.xe(x))
        return F.lstm(c, self.eh(e) + self.hh(h))

class LSTM_Decoder(chainer.Chain):
    def __init__(self, vocab_size, embed_size, hidden_size):
        """Decoderのインスタンス化
        
        Args:
            vocab_size: 使われる単語の種類数（語彙数）
            embed_size: 単語をベクトル表現した際のサイズ
            hidden_size: 隠れ層のサイズ
        """
        super(LSTM_Decoder, self).__init__(
            ye = L.EmbedID(vocab_size, embed_size, ignore_label=-1),
            eh = L.Linear(embed_size, 4 * hidden_size),
            hh = L.Linear(hidden_size, 4 * hidden_size),
            he = L.Linear(hidden_size, embed_size),
            ey = L.Linear(embed_size, vocab_size)
        )

    def __call__(self, y, c, h):
        """Decoderの計算
        
        Args:
            y: one-hotな単語
            c: 内部メモリ
            h: 隠れ層
        Returns:
            予測単語、次の内部メモリ、次の隠れ層
        """
        e = F.tanh(self.ye(y))
        c, h = F.lstm(c, self.eh(e) + self.hh(h))
        t = self.ey(F.tanh(self.he(h)))
        return t, c, h
    
class Seq2Seq(chainer.Chain):
    def __init__(self, vocab_size, embed_size, hidden_size):
        """Seq2Seqのインスタンス化
        
        Args:
            vocab_size: 語彙サイズ
            embed_size: 単語ベクトルのサイズ
            hidden_size: 中間ベクトルのサイズ
        """
        super(Seq2Seq, self).__init__(
            encoder = LSTM_Encoder(vocab_size, embed_size, hidden_size), # Encoderのインスタンス化
            decoder = LSTM_Decoder(vocab_size, embed_size, hidden_size) # Decoderのインスタンス化
        )
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.decode_max_size = 20 # デコードはEOSが出力されれば終了する、出力されない場合の最大出力語彙数
        
    def encode(self, words, batch_size):
        """Encoderの計算
        
        Args:
            words: 単語が記録されたリスト
        """
        # 内部メモリ、中間ベクトルの初期化
        c = chainer.Variable(xp.zeros((batch_size, self.hidden_size), dtype='float32'))
        h = chainer.Variable(xp.zeros((batch_size, self.hidden_size), dtype='float32'))
        # エンコーダーに単語を順番に読み込ませる
        for w in words:
            c, h = self.encoder(w, c, h)
        self.h = h # 計算した中間ベクトルをデコーダーに引き継ぐためにインスタンス変数にする
        self.c = chainer.Variable(xp.zeros((batch_size, self.hidden_size), dtype='float32')) # 内部メモリは引き継がないので、初期化

    def decode(self, w):
        """Decoderの計算
        
        Args:
            w: 単語
        Returns:
            単語数サイズのベクトル
        """
        t, self.c, self.h = self.decoder(w, self.c, self.h)
        return t

    def reset(self):
        """勾配の初期化
        """
        self.zerograds()
    
    def __call__(self, enc_words, dec_words=None, train=True):
        """順伝播の計算を行う関数
        
        Args:
            enc_words: 発話文の単語を記録したリスト
            dec_words: 応答文の単語を記録したリスト
        Returns:
            計算した損失の合計 or 予測したデコード文字列
        """
        enc_words = enc_words.T
        if train:
            dec_words = dec_words.T
        batch_size = len(enc_words[0]) # バッチサイズを記録
        self.reset() # model内に保存されている勾配をリセット
        enc_words = [chainer.Variable(xp.array(row, dtype='int32')) for row in enc_words] # 発話リスト内の単語を、chainerの型であるVariable型に変更
        self.encode(enc_words, batch_size) # エンコードの計算
        loss = chainer.Variable(xp.zeros((), dtype='float32')) # 損失の初期化
        t = chainer.Variable(xp.array([0 for _ in range(batch_size)], dtype='int32')) # <eos>をデコーダーに読み込ませる
        ys = [] # デコーダーが生成するデコード文字列を格納する配列
        # デコーダーの計算
        if train: # 学習の場合は損失を計算する
            for w in dec_words:
                y = self.decode(t) # 1単語ずつをデコードする
                t = chainer.Variable(xp.array(w, dtype='int32')) # 正解単語をVariable型に変換
                loss += F.softmax_cross_entropy(y, t) # 正解単語と予測単語を照らし合わせて損失を計算
                print(loss)
            return loss
        else: # 予測の場合はデコード文字列を生成する
            for i in range(self.decode_max_size):
                y = self.decode(t)
                y = np.argmax(y.data) # 確率で出力されたままなので、確率が高い予測単語を取得する
                ys.append(y)
                t = chainer.Variable(xp.array([y], dtype='int32'))
                if y == 0: # EOSを出力したならばデコードを終了する
                    break
            return ys

In [13]:
# 学習

# 定数
embed_size = 100
hidden_size = 100
batch_col_size = 15
batch_size = 6 # ミニバッチ学習のバッチサイズ数
epoch_num = 50 # エポック数
N = len(data) # 教師データの数

# 教師データの読み込み
data_converter = DataConverter(batch_col_size=batch_col_size) # データコンバーター
data_converter.load(data) # 教師データ読み込み
vocab_size = len(data_converter.vocab) # 単語数

# モデルの宣言
model = Seq2Seq(vocab_size=vocab_size, embed_size=embed_size, hidden_size=hidden_size)
opt = chainer.optimizers.Adam()
opt.setup(model)
opt.add_hook(chainer.optimizer.GradientClipping(5))

if gpu >= 0:
    model.to_gpu(gpu)

model.reset()

In [14]:
# 学習

st = datetime.datetime.now()

# 学習開始
for epoch in range(epoch_num):
    
    # ミニバッチ学習
    perm = np.random.permutation(N) # ランダムな整数列リストを取得
    total_loss = 0
    
    for i in range(0, N, batch_size):
        enc_words = data_converter.train_queries[perm[i:i+batch_size]]
        dec_words = data_converter.train_responses[perm[i:i+batch_size]]
        model.reset()
        loss = model(enc_words=enc_words, dec_words=dec_words, train=True)
        loss.backward()
        loss.unchain_backward()
        total_loss += loss.data
        opt.update()
        
    if (epoch+1)%10 == 0:
        ed = datetime.datetime.now()
        print("epoch:\t{}\ttotal loss:\t{}\ttime:\t{}".format(epoch+1, total_loss, ed-st))
        st = datetime.datetime.now()

variable(4.207329750061035)
variable(8.475435256958008)
variable(12.722667694091797)
variable(16.97272491455078)
variable(21.358600616455078)
variable(25.519550323486328)
variable(29.8553524017334)
variable(33.98801040649414)
variable(38.36465072631836)
variable(42.41445541381836)
variable(42.41445541381836)
variable(42.41445541381836)
variable(42.41445541381836)
variable(42.41445541381836)
variable(42.41445541381836)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)

/Users/kazuki_tanimori/anaconda3/lib/python3.6/site-packages/chainer/optimizer_hooks/gradient_clipping.py:54: RuntimeWarning: invalid value encountered in sqrt
  norm = cuda.get_array_module(sqnorm).sqrt(sqnorm)
/Users/kazuki_tanimori/anaconda3/lib/python3.6/site-packages/chainer/optimizer_hooks/gradient_clipping.py:55: RuntimeWarning: divide by zero encountered in double_scalars
  rate = self.threshold / norm



variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
varia

variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variab

variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variable(nan)
variab

In [18]:
def predict(model, query):
    enc_query = data_converter.sentence2ids(query, train=False)
    dec_response = model(enc_words=enc_query, train=False)
    response = data_converter.ids2words(dec_response)
    print(query, "=>", response)

predict(model, "初めまして。")
predict(model, "どこから来たんですか？")
predict(model, "日本のどこに住んでるんですか？")
predict(model, "仕事は何してますか？")
predict(model, "お会いできて嬉しかったです。")
predict(model, "おはよう。")
predict(model, "いつも何時に起きますか？")
predict(model, "朝食は何を食べますか？")
predict(model, "朝食は毎日食べますか？")
predict(model, "野菜をたくさん取っていますか？")
predict(model, "週末は何をしていますか？")
predict(model, "どこに行くのが好き？")

初めまして。 => ['<eos>']
どこから来たんですか？ => ['<eos>']
日本のどこに住んでるんですか？ => ['<eos>']
仕事は何してますか？ => ['<eos>']
お会いできて嬉しかったです。 => ['<eos>']
おはよう。 => ['<eos>']
いつも何時に起きますか？ => ['<eos>']
朝食は何を食べますか？ => ['<eos>']
朝食は毎日食べますか？ => ['<eos>']
野菜をたくさん取っていますか？ => ['<eos>']
週末は何をしていますか？ => ['<eos>']
どこに行くのが好き？ => ['<eos>']


In [18]:
!python --version

Python 3.6.3 :: Anaconda, Inc.


In [19]:
!pip freeze

alabaster==0.7.10
anaconda-client==1.6.5
anaconda-navigator==1.6.9
anaconda-project==0.8.0
appnope==0.1.0
appscript==1.0.1
asn1crypto==0.22.0
astroid==1.5.3
astropy==2.0.2
Babel==2.5.0
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.6.0
bitarray==0.8.1
bkcharts==0.2
blaze==0.11.3
bleach==2.0.0
bokeh==0.12.10
boto==2.48.0
boto3==1.9.2
botocore==1.12.2
Bottleneck==1.2.1
bz2file==0.98
certifi==2017.7.27.1
cffi==1.10.0
chainer==4.4.0
chardet==3.0.4
click==6.7
cloudpickle==0.4.0
clyent==1.2.2
colorama==0.3.9
conda==4.3.30
conda-build==3.0.27
conda-verify==2.0.0
contextlib2==0.5.5
cryptography==2.0.3
cycler==0.10.0
Cython==0.26.1
cytoolz==0.8.2
dask==0.15.3
datashape==0.5.4
decorator==4.1.2
distributed==1.19.1
docutils==0.14
entrypoints==0.2.3
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==2.0.12
Flask==0.12.2
Flask-Cors==3.0.3
gensim==3.5.0
gevent==1.2.2
glob2==0.5
gmpy2==2.0.8
greenlet==0.4.12
h5py==2.7.0
heapdict==1.0.0
html5lib==0.999999999
idna==2.6
imageio==2.2.0
imagesize==0